### Time Intro
> The time module provides access to several different types of clocks, each useful for different purposes. The standard system calls like time() report the system “wall clock” time. The monotonic() clock can be used to measure elapsed time in a long-running process because it is guaranteed never to move backwards, even if the system time is changed. For performance testing, perf_counter() provides access to the clock with the highest available resolution to make short time measurements more accurate. The CPU time is available through clock(), and process_time() returns the combined processor time and system time.

- time()
> 提供系统时间
- monotonic()
> 测量长时间运行的程序的经过时间
- perf_counter()
> 用于短时间性能测试,可供可用的分辨率最高的时间测量
- clock()
> CPU时间
- process_time()
> 特定处理器的时间

**Note**
> The implementations expose C library functions for manipulating dates and times. Since they are tied to the underlying C implementation, some details (such as the start of the epoch and maximum date value supported) are platform-specific. Refer to the library documentation for complete details.

### Compare Clocks
> Implementation details for the clocks varies by platform. Use get_clock_info() to access basic information about the current implementation, including the clock’s resolution.

In [2]:
# get clock 
import textwrap
import time

available_clocks = [
    ('monotonic', time.monotonic),
    ('perf_counter', time.perf_counter),
    ('process_time', time.process_time),
    ('time', time.time),
]

for clock_name, func in available_clocks:
    print(textwrap.dedent('''\
    {name}:
        adjustable    : {info.adjustable}
        implementation: {info.implementation}
        monotonic     : {info.monotonic}
        resolution    : {info.resolution}
        current       : {current}
    ''').format(
        name=clock_name,
        info=time.get_clock_info(clock_name),
        current=func())
    )

monotonic:
    adjustable    : False
    implementation: clock_gettime(CLOCK_MONOTONIC)
    monotonic     : True
    resolution    : 1e-09
    current       : 140456.542205564

perf_counter:
    adjustable    : False
    implementation: clock_gettime(CLOCK_MONOTONIC)
    monotonic     : True
    resolution    : 1e-09
    current       : 140456.542502828

process_time:
    adjustable    : False
    implementation: clock_gettime(CLOCK_PROCESS_CPUTIME_ID)
    monotonic     : True
    resolution    : 1e-09
    current       : 0.419968712

time:
    adjustable    : True
    implementation: clock_gettime(CLOCK_REALTIME)
    monotonic     : False
    resolution    : 1e-09
    current       : 1589448532.8935058



### Wall Clock TIme
> One of the core functions of the time module is time(), which returns the number of seconds since the start of the “epoch” as a floating point value.
> 
> 返回自公元纪年开始的秒数,浮点数

In [4]:
import time
print("The time is ",time.time())

# For logging or printing time ctime() can be more useful.
# It is useful for producing human readable representations.
print("The time is ",time.ctime())

The time is  1589448754.0994627
The time is  Thu May 14 17:32:34 2020


### Monotonic Clocks
> Because time() looks at the system clock, and the system clock can be changed by the user or system services for synchronizing clocks across multiple computers, calling time() repeatedly may produce values that go forwards and backwards. This can result in unexpected behavior when trying to measure durations or otherwise use those times for computation. Avoid those situations by using monotonic(), which always returns values that go forward.
> 
> time()依赖于系统时间,而系统时间可以被修改.为了准确测量时间段,使用单调时钟的monotonic(),单调时钟的起点是未定义的,因此,两次monotonic()之间的差值才有意义.

In [6]:
import time

start = time.monotonic()
time.sleep(0.1)
end = time.monotonic()
print('start : {:>9.2f}'.format(start))
print('end   : {:>9.2f}'.format(end))
print('span  : {:>9.2f}'.format(end - start))

start : 140988.78
end   : 140988.89
span  :      0.10


### Processor Clock Time
> While time() returns a wall clock time, process_time() returns processor clock time. The values returned from process_time() reflect the actual time used by the program as it runs.
>
> process_time() 返回处理器的时间

In [9]:
import hashlib
import time

# Data to use to calculate md5 checksums
data = "hello,world.".encode("utf-8")

for i in range(5):
    h = hashlib.sha1()
    print(time.ctime(), ': {:0.3f} {:0.3f}'.format(
        time.time(), time.process_time()))
    for i in range(300000):
        h.update(data)
    cksum = h.digest()

Thu May 14 17:41:44 2020 : 1589449304.901 1.424
Thu May 14 17:41:44 2020 : 1589449304.936 1.461
Thu May 14 17:41:44 2020 : 1589449304.968 1.492
Thu May 14 17:41:44 2020 : 1589449304.998 1.523
Thu May 14 17:41:45 2020 : 1589449305.028 1.553


In [10]:
# the processor clock does not tick 
# if a program is not doing anything.
# The time() value increases even while the application is asleep, 
# but the process_time() value does not.
import time

template = '{} - {:0.2f} - {:0.2f}'

print(template.format(
    time.ctime(), time.time(), time.process_time())
)

for i in range(3, 0, -1):
    print('Sleeping', i)
    time.sleep(i)
    print(template.format(
        time.ctime(), time.time(), time.process_time())
    )

Thu May 14 17:44:46 2020 - 1589449486.66 - 1.60
Sleeping 3
Thu May 14 17:44:49 2020 - 1589449489.66 - 1.60
Sleeping 2
Thu May 14 17:44:51 2020 - 1589449491.66 - 1.61
Sleeping 1
Thu May 14 17:44:52 2020 - 1589449492.66 - 1.61


### Performance Counter
> It is important to have a high-resolution monotonic clock for measuring performance. Determining the best clock data source requires platform-specific knowledge, which Python provides in perf_counter().
> 
> 返回一个最高分辨率的时钟，和monotonic()一样，起始时钟未定义，两次调用的差值才是有意义的数据。

In [11]:
import hashlib
import time

# Data to use to calculate md5 checksums
data = "hello,world.".encode("utf-8")

loop_start = time.perf_counter()

for i in range(5):
    iter_start = time.perf_counter()
    h = hashlib.sha1()
    for i in range(300000):
        h.update(data)
    cksum = h.digest()
    now = time.perf_counter()
    loop_elapsed = now - loop_start
    iter_elapsed = now - iter_start
    print(time.ctime(), ': {:0.3f} {:0.3f}'.format(
        iter_elapsed, loop_elapsed))

Thu May 14 17:50:08 2020 : 0.046 0.047
Thu May 14 17:50:08 2020 : 0.029 0.076
Thu May 14 17:50:08 2020 : 0.029 0.105
Thu May 14 17:50:08 2020 : 0.029 0.134
Thu May 14 17:50:08 2020 : 0.029 0.163
